In [1]:
import numpy as np
import pandas as pd
import os 

In [2]:
xtr = pd.read_csv("C:/Users/Rohit/Downloads/titanic/train.csv")
xt = pd.read_csv("C:/Users/Rohit/Downloads/titanic/test.csv")


In [3]:
xtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [4]:
xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 27.8+ KB


In [5]:
xtr.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
Target = 'Survived'

xtr.dropna(axis=0, subset=[Target], inplace=True)

In [7]:
col = ["Ticket","Cabin","Name","PassengerId","Embarked"]
xt = xt.drop(col,axis =1)
xtr = xtr.drop(col,axis =1)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
xtr["Sex"] = le.fit_transform(xtr["Sex"])
xt["Sex"] = le.fit_transform(xt["Sex"])

In [9]:
xtr = xtr.fillna(xtr["Age"].mean())
xt = xt.fillna(xt["Age"].mean())

In [10]:
xtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 52.2 KB


In [11]:
xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    int32  
 2   Age     418 non-null    float64
 3   SibSp   418 non-null    int64  
 4   Parch   418 non-null    int64  
 5   Fare    418 non-null    float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 18.0 KB


In [12]:
inp = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
out = ["Survived"]

In [13]:
def entropy(col):
    e = 0.0
    count = np.unique(col,return_counts = True)
    n = float(col.shape[0])
    for i in count[1]:
        p = i/n
        e += (-1.0*p*np.log2(p))
    return e


In [14]:
def divide(xdata,fkey,fval):
    xr = pd.DataFrame([],columns = xdata.columns)
    xl = pd.DataFrame([],columns = xdata.columns)
    for i in range(xdata.shape[0]):
        val = xdata[fkey].loc[i]
        if val> fval:
            xr = xr.append(xdata.loc[i])
        else:
            xl = xl.append(xdata.loc[i])
    return xr,xl
        

In [15]:
def infogain(xdata,fkey,fval):
    right , left = divide(xdata,fkey,fval)
    l = float(left.shape[0])/xdata.shape[0]
    r = float(right.shape[0])/xdata.shape[0]
    
    if left.shape[0] ==0 or right.shape[0] == 0:
        return -100000
    ig = entropy(xdata.Survived) - (l*entropy(left.Survived ) + r*entropy(right.Survived))
    return ig

In [16]:
class DT:
    def __init__(self,dep = 0,maxd = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.target = None
        self.depth = dep
        self.maxd = maxd
        
    def train(self,xtr):
        ft = ['Pclass','Sex','Age','SibSp','Parch','Fare']
        igg = []
        for i in ft:
            ig = infogain(xtr,i,xtr[i].mean())
            igg.append(ig)

        self.fkey = ft[np.argmax(igg)]
        self.fval = xtr[self.fkey].mean()
        print(self.fkey)
        
        rr , ll = divide(xtr,self.fkey,self.fval)
        
        rr = rr.reset_index(drop= True)
        ll = ll.reset_index(drop = True)
        
        if ll.shape[0] ==0 or rr.shape[0] == 0:
            if xtr.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        if (self.depth >= self.maxd):
            if xtr.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        self.left = DT(dep = self.depth +1,maxd = self.maxd )
        self.left.train(ll)
        self.right = DT(dep = self.depth +1,maxd = self.maxd )
        self.right.train(rr)
        
        if (self.depth >= self.maxd):
            if xtr.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return 
        
    def predict(self,test):
        print(self.fkey,test,self.fval)
        print("-------")
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)    

In [17]:
dd = DT()
dd.train(xtr)

Sex
Pclass
Pclass
Parch
Age
Age
Age
Parch
Age
Fare
Parch
Age
Age
Age
Age
Age
Age
Fare
SibSp
Fare
Fare
Age
SibSp
Parch
Age
SibSp
Fare
Parch
Parch
Age
Age
Age
Fare
Parch
Pclass
Pclass
Age
SibSp
Age
Age
Age
Age
Age
SibSp
Age
Age
Fare
Age
Pclass
Age
Age
Fare
Age
Age
Parch
SibSp
SibSp
Pclass
Parch
SibSp
Parch
Age
Age


In [18]:
ypred = []
for ix in range(xt.shape[0]):
    ypred.append(dd.predict(xt.loc[ix]))
print(ypred)

Sex Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 0.6475869809203143
-------
Fare Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 25.523893414211443
-------
Parch Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 0.08883248730964467
-------
Pclass Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 2.7774725274725274
-------
Age Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 29.060646244657033
-------
Age Pclass     3.0000
Sex        1.0000
Age       34.5000
SibSp      0.0000
Parch      0.0000
Fare       7.8292
Name: 0, dtype: float64 34.36891727941176
-------
Sex Pclass     3

Sex Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 0.6475869809203143
-------
Fare Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 25.523893414211443
-------
Parch Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 0.08883248730964467
-------
Pclass Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 2.7774725274725274
-------
Age Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 29.060646244657033
-------
Age Pclass     3.0
Sex        1.0
Age       25.0
SibSp      0.0
Parch      0.0
Fare       9.5
Name: 38, dtype: float64 22.476744186046513
-------
Sex Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare      56.49580

Name: 73, dtype: float64 39.51468306288032
-------
Age Pclass     1.0000
Sex        1.0000
Age       28.5000
SibSp      0.0000
Parch      0.0000
Fare      27.7208
Name: 73, dtype: float64 28.409425551470587
-------
Age Pclass     1.0000
Sex        1.0000
Age       28.5000
SibSp      0.0000
Parch      0.0000
Fare      27.7208
Name: 73, dtype: float64 32.177639885222376
-------
Sex Pclass      1.0
Sex         0.0
Age        35.0
SibSp       0.0
Parch       0.0
Fare      211.5
Name: 74, dtype: float64 0.6475869809203143
-------
Pclass Pclass      1.0
Sex         0.0
Age        35.0
SibSp       0.0
Parch       0.0
Fare      211.5
Name: 74, dtype: float64 2.159235668789809
-------
Pclass Pclass      1.0
Sex         0.0
Age        35.0
SibSp       0.0
Parch       0.0
Fare      211.5
Name: 74, dtype: float64 1.4470588235294117
-------
Parch Pclass      1.0
Sex         0.0
Age        35.0
SibSp       0.0
Parch       0.0
Fare      211.5
Name: 74, dtype: float64 0.4574468085106383
-------
Age Pc

Parch Pclass     3.0000
Sex        1.0000
Age       18.0000
SibSp      1.0000
Parch      0.0000
Fare      14.4542
Name: 115, dtype: float64 0.08883248730964467
-------
Pclass Pclass     3.0000
Sex        1.0000
Age       18.0000
SibSp      1.0000
Parch      0.0000
Fare      14.4542
Name: 115, dtype: float64 2.7774725274725274
-------
Age Pclass     3.0000
Sex        1.0000
Age       18.0000
SibSp      1.0000
Parch      0.0000
Fare      14.4542
Name: 115, dtype: float64 29.060646244657033
-------
Age Pclass     3.0000
Sex        1.0000
Age       18.0000
SibSp      1.0000
Parch      0.0000
Fare      14.4542
Name: 115, dtype: float64 22.476744186046513
-------
Sex Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       6.43750
Name: 116, dtype: float64 0.6475869809203143
-------
Fare Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       6.43750
Name: 116, dtype: float64 25.523893414211443

Name: 147, dtype: float64 2.7774725274725274
-------
Age Pclass     3.00
Sex        1.00
Age       22.00
SibSp      0.00
Parch      0.00
Fare       8.05
Name: 147, dtype: float64 29.060646244657033
-------
Age Pclass     3.00
Sex        1.00
Age       22.00
SibSp      0.00
Parch      0.00
Fare       8.05
Name: 147, dtype: float64 22.476744186046513
-------
Sex Pclass     1.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare      26.55000
Name: 148, dtype: float64 0.6475869809203143
-------
Fare Pclass     1.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare      26.55000
Name: 148, dtype: float64 25.523893414211443
-------
Pclass Pclass     1.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare      26.55000
Name: 148, dtype: float64 1.540983606557377
-------
Age Pclass     1.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare      26.55000
Name: 1

...

Fare Pclass     3.0000
Sex        1.0000
Age       27.0000
SibSp      0.0000
Parch      0.0000
Fare       7.8958
Name: 334, dtype: float64 25.523893414211443
-------
Parch Pclass     3.0000
Sex        1.0000
Age       27.0000
SibSp      0.0000
Parch      0.0000
Fare       7.8958
Name: 334, dtype: float64 0.08883248730964467
-------
Pclass Pclass     3.0000
Sex        1.0000
Age       27.0000
SibSp      0.0000
Parch      0.0000
Fare       7.8958
Name: 334, dtype: float64 2.7774725274725274
-------
Age Pclass     3.0000
Sex        1.0000
Age       27.0000
SibSp      0.0000
Parch      0.0000
Fare       7.8958
Name: 334, dtype: float64 29.060646244657033
-------
Age Pclass     3.0000
Sex        1.0000
Age       27.0000
SibSp      0.0000
Parch      0.0000
Fare       7.8958
Name: 334, dtype: float64 22.476744186046513
-------
Sex Pclass     1.0
Sex        1.0
Age       30.0
SibSp      0.0
Parch      0.0
Fare      26.0
Name: 335, dtype: float64 0.6475869809203143
-------
Fare Pclass     1.0
S

Name: 357, dtype: float64 25.523893414211443
-------
Parch Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.87920
Name: 357, dtype: float64 0.08883248730964467
-------
Pclass Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.87920
Name: 357, dtype: float64 2.7774725274725274
-------
Age Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.87920
Name: 357, dtype: float64 29.060646244657033
-------
Age Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.87920
Name: 357, dtype: float64 34.36891727941176
-------
Sex Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.75000
Name: 358, dtype: float64 0.6475869809203143
-------
Fare Pclass     3.00000
Sex        1.00000
Age       30.27259
SibSp      0.00000
Parch 

Name: 373, dtype: float64 0.08883248730964467
-------
Pclass Pclass     2.0
Sex        1.0
Age       44.0
SibSp      0.0
Parch      0.0
Fare      13.0
Name: 373, dtype: float64 2.7774725274725274
-------
Pclass Pclass     2.0
Sex        1.0
Age       44.0
SibSp      0.0
Parch      0.0
Fare      13.0
Name: 373, dtype: float64 1.92
-------
SibSp Pclass     2.0
Sex        1.0
Age       44.0
SibSp      0.0
Parch      0.0
Fare      13.0
Name: 373, dtype: float64 0.07246376811594203
-------
Sex Pclass     1.0000
Sex        0.0000
Age       54.0000
SibSp      1.0000
Parch      1.0000
Fare      81.8583
Name: 374, dtype: float64 0.6475869809203143
-------
Pclass Pclass     1.0000
Sex        0.0000
Age       54.0000
SibSp      1.0000
Parch      1.0000
Fare      81.8583
Name: 374, dtype: float64 2.159235668789809
-------
Pclass Pclass     1.0000
Sex        0.0000
Age       54.0000
SibSp      1.0000
Parch      1.0000
Fare      81.8583
Name: 374, dtype: float64 1.4470588235294117
-------
Parch Pcla

Fare Pclass     3.000
Sex        1.000
Age       29.000
SibSp      3.000
Parch      1.000
Fare      22.025
Name: 394, dtype: float64 25.523893414211443
-------
Parch Pclass     3.000
Sex        1.000
Age       29.000
SibSp      3.000
Parch      1.000
Fare      22.025
Name: 394, dtype: float64 0.08883248730964467
-------
Age Pclass     3.000
Sex        1.000
Age       29.000
SibSp      3.000
Parch      1.000
Fare      22.025
Name: 394, dtype: float64 19.340549019607845
-------
Age Pclass     3.000
Sex        1.000
Age       29.000
SibSp      3.000
Parch      1.000
Fare      22.025
Name: 394, dtype: float64 32.664033613445376
-------
Fare Pclass     3.000
Sex        1.000
Age       29.000
SibSp      3.000
Parch      1.000
Fare      22.025
Name: 394, dtype: float64 18.175
-------
Sex Pclass     1.0
Sex        0.0
Age       18.0
SibSp      1.0
Parch      0.0
Fare      60.0
Name: 395, dtype: float64 0.6475869809203143
-------
Pclass Pclass     1.0
Sex        0.0
Age       18.0
SibSp      1.

Name: 408, dtype: float64 2.159235668789809
-------
Fare Pclass     3.00000
Sex        0.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.72080
Name: 408, dtype: float64 16.118809722222224
-------
SibSp Pclass     3.00000
Sex        0.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.72080
Name: 408, dtype: float64 0.30927835051546393
-------
Fare Pclass     3.00000
Sex        0.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.72080
Name: 408, dtype: float64 8.973615277777778
-------
Fare Pclass     3.00000
Sex        0.00000
Age       30.27259
SibSp      0.00000
Parch      0.00000
Fare       7.72080
Name: 408, dtype: float64 7.7934792452830175
-------
Sex Pclass     3.000
Sex        0.000
Age        3.000
SibSp      1.000
Parch      1.000
Fare      13.775
Name: 409, dtype: float64 0.6475869809203143
-------
Pclass Pclass     3.000
Sex        0.000
Age        3.000
SibSp      1.000
Parch      1.000
Fare    

In [19]:
from sklearn.tree import DecisionTreeClassifier
sk = DecisionTreeClassifier(max_depth = 5)
sk.fit(xtr[inp],xtr[out])
sk.score(xtr[inp],xtr[out])

0.8417508417508418

In [20]:
pred = sk.predict(xt[inp])

In [21]:
type(pred)
print(pred)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1
 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [24]:
df = pd.DataFrame(pred,columns = ["Survived"])

In [25]:
df.to_csv("pred.csv")